In [1]:


#CONSTRUCT CLASSES

class Data:      
    label = 0
    joints=[]
    filename =''
    random_array = []
    frame = 0
    skelno = 0


In [2]:
def read_skeleton(filename,count):
    with open(filename) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
  
    frame_number = int(content[0])
    print("Frame Number: ",frame_number)
    #print(len(content))
    #print(content[2])
    
    
    new_skel = Data()
    
    xyz_skeletons = []
    
    new_skel.frame = frame_number



    
    joint_number = 25
    skeleton2 = 2
    skeleton3 = 3
    
    
    if(int(content[1])==1):
        a=4
        b=0
        new_skel.skelno = 1
        new_skel.joints =  np.zeros(((frame_number, 25, 2, 3)))
        new_skel.label = int(filename[18:20])
        for i in range(frame_number):
            for j in range(joint_number):
                splitted_data = content[a+j].split()
                new_skel.joints[i,j,0,0] = splitted_data[0]
                new_skel.joints[i,j,0,1] = splitted_data[1]
                new_skel.joints[i,j,0,2] = splitted_data[2]
                new_skel.filename = filename
            a=a+28
            b=b+3
        xyz_skeletons.append(new_skel)
        
        
    if(int(content[1])==2):       
        a=4
        b=0
        new_skel.skelno = 2
        new_skel.joints =  np.zeros(((frame_number, 25, 2, 3)))
        new_skel.label = int(filename[18:20])
        for i in range(frame_number):
            for s in range(skeleton2):
                for j in range(joint_number):
                    splitted_data = content[a+j].split()
                    new_skel.joints[i,j,s,0] = splitted_data[0]
                    new_skel.joints[i,j,s,1] = splitted_data[1]
                    new_skel.joints[i,j,s,2] = splitted_data[2]
                    new_skel.filename = filename
                if (s==0):
                    a=a+27
            b=b+3 
            a=a+28           
        xyz_skeletons.append(new_skel)
        
        
        
        
    if(int(content[1])==3):
                
        a=4
        b=0
        new_skel.joints =  np.zeros((75, 3*frame_number))
        new_skel.label = int(filename[18:20])
        for i in range(frame_number):
            for s in range(skeleton3):
                for j in range(joint_number):
                    splitted_data = content[a+j].split()
                    new_skel.joints[j + s*25][i][0] = splitted_data[0]
                    new_skel.joints[j + s*25][i][1] = splitted_data[1] 
                    new_skel.joints[j + s*25][i][2] = splitted_data[2] 
                    new_skel.filename = filename
                if (s<=1):
                    a=a+27
            a=a+28
            b=b+3
        xyz_skeletons.append(new_skel)
        
        
            
        
    return new_skel       
        

In [ ]:
import glob, os
import csv
from itertools import islice
import numpy as np
import math
import pickle

os.chdir("/home/murat/Desktop/skeleton_s1/s001")
count=0


skeleton_data = []

for filename in glob.glob("*.skeleton"): 
    
    count = count + 1
    print("++++++++++++++++++++++++++++")
    print(count,filename)
    print("++++++++++++++++++++++++++++")
    skeleton_data.append(read_skeleton(filename,count))

os.chdir("/home/murat/Desktop/ebbt_har/compact_data")    
pickle.dump(skeleton_data, open('skeleton_data', 'wb')) 

In [9]:

#reconstruct data with reference skeleton number 2  

joint_number = 25
skeleton_number = 2 

for i in range(len(skeleton_data)):
    for j in range(skeleton_data[i].joints.shape[0]):
        for k in range(joint_number):
            if (k!=1):
                for s in range(skeleton_number):                                                          
                    skeleton_data[i].joints[j,1,s,2]
                    skeleton_data[i].joints[j,k,s,0] = skeleton_data[i].joints[j,k,s,0] - skeleton_data[i].joints[j,1,s,0]
                    skeleton_data[i].joints[j,k,s,1] = skeleton_data[i].joints[j,k,s,1] - skeleton_data[i].joints[j,1,s,1]
                    skeleton_data[i].joints[j,k,s,2] = skeleton_data[i].joints[j,k,s,2] - skeleton_data[i].joints[j,1,s,2]                 
        skeleton_data[i].joints[j,1,1,0] = 0
        skeleton_data[i].joints[j,1,1,1] = 0
        skeleton_data[i].joints[j,1,1,2] = 0
        skeleton_data[i].joints[j,1,0,0] = 0
        skeleton_data[i].joints[j,1,0,1] = 0
        skeleton_data[i].joints[j,1,0,2] = 0


            


In [16]:
np.mean(skeleton_data[0].joints[:,3,0,:], axis=0)

array([-0.00071687,  0.39823454, -0.08065886])

In [17]:
##########  NORMALIZE DATA 

joint_number = 25

for i in range(len(skeleton_data)):     
    for j in range(joint_number):
        mean_xyz = np.mean(skeleton_data[i].joints[:,j,0,:], axis=0)
        std_xyz = np.std(skeleton_data[i].joints[:,j,0,:], axis=0)
        for k in range(skeleton_data[i].joints.shape[0]):
            skeleton_data[i].joints[k,j,0,0] = (skeleton_data[i].joints[k,j,0,0] - mean_xyz[0]) / std_xyz[0]
            skeleton_data[i].joints[k,j,0,1] = (skeleton_data[i].joints[k,j,0,1] - mean_xyz[1]) / std_xyz[1]
            skeleton_data[i].joints[k,j,0,2] = (skeleton_data[i].joints[k,j,0,2] - mean_xyz[2]) / std_xyz[2]
            skeleton_data[i].joints[k,1,0,0] = 0
            skeleton_data[i].joints[k,1,0,1] = 0
            skeleton_data[i].joints[k,1,0,2] = 0

    
    
    if(skeleton_data[i].skelno == 2):
        for j in range(joint_number):
            mean_xyz = np.mean(skeleton_data[i].joints[:,j,1,:], axis=0)
            std_xyz = np.std(skeleton_data[i].joints[:,j,1,:], axis=0)
            for k in range(skeleton_data[i].joints.shape[0]):
                skeleton_data[i].joints[k,j,1,0] = (skeleton_data[i].joints[k,j,1,0] - mean_xyz[0]) / std_xyz[0]
                skeleton_data[i].joints[k,j,1,1] = (skeleton_data[i].joints[k,j,1,1] - mean_xyz[1]) / std_xyz[1]
                skeleton_data[i].joints[k,j,1,2] = (skeleton_data[i].joints[k,j,1,2] - mean_xyz[2]) / std_xyz[2]
                skeleton_data[i].joints[k,1,1,0] = 0
                skeleton_data[i].joints[k,1,1,1] = 0
                skeleton_data[i].joints[k,1,1,2] = 0
    mean_xyz = 0
    std_xyz = 0

/home/murat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/home/murat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/home/murat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/home/murat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars
/home/murat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
/home/murat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


In [24]:
#choose random 100 frame of the video

sequence_number = 100
joint_number = 25
skeleton_number = 2

sk = []


for i in range(len(skeleton_data)):
    
    seq = Data()
    
    seq.joints =  np.zeros(((sequence_number,25,2,3)))
    
    if (skeleton_data[i].frame <= sequence_number):
        rd = np.arange(skeleton_data[i].frame)
        for j in range(skeleton_data[i].frame):
            for k in range(joint_number):
                for m in range(skeleton_number):
                    seq.joints[j + int((sequence_number - skeleton_data[i].frame)/2), k, m, 0] = skeleton_data[i].joints[rd[j], k, m,0]
                    seq.joints[j + int((sequence_number - skeleton_data[i].frame)/2), k, m, 1] = skeleton_data[i].joints[rd[j], k, m,1]
                    seq.joints[j + int((sequence_number - skeleton_data[i].frame)/2), k, m, 2] = skeleton_data[i].joints[rd[j], k, m,2]
                    seq.filename = skeleton_data[i].filename 
                    seq.label = skeleton_data[i].label
                    seq.random_array = rd
                    seq.frame = skeleton_data[i].frame
        sk.append(seq)
        
    else:
        rd = np.random.choice(skeleton_data[i].frame, sequence_number, replace=False)
        rd.sort()
        for j in range(sequence_number):
            for k in range(joint_number):
                for m in range(skeleton_number):
                    seq.joints[j,k,m,0] = skeleton_data[i].joints[rd[j],k,m,0]
                    seq.joints[j,k,m,1] = skeleton_data[i].joints[rd[j],k,m,1]
                    seq.joints[j,k,m,2] = skeleton_data[i].joints[rd[j],k,m,2]
                    seq.filename = skeleton_data[i].filename 
                    seq.label = skeleton_data[i].label
                    seq.random_array = rd
                    seq.frame = skeleton_data[i].frame
        sk.append(seq)
    
os.chdir("/home/murat/Desktop/ebbt_har/compact_data")    
pickle.dump(sk, open('sk', 'wb'))   

# minimum frame number


m = sk[0].frame
for i in range(len(sk)):
    if (sk[i].frame<=m):
        m = sk[i].frame
print("minimum frame number: ",m)

FileNotFoundError: [Errno 2] No such file or directory: '/home/murat/Desktop/ebbt_har/compact_data'

In [29]:
train = sk[:5000]
test =  sk[5000:]
print('train joint', train[0].joints.shape)
print('test joint', test[0].joints.shape)

train joint (100, 25, 2, 3)
test joint (100, 25, 2, 3)


In [30]:
#construct one hot vectors


train_data = np.zeros(((((len(train),sequence_number,joint_number,2,3)))))
train_label = np.zeros((len(train),60))


for i in range(len(train)):
    for j in range(train[i].joints.shape[0]):
        for k in range(train[i].joints.shape[1]):
            for m in range(skeleton_number):
                train_data[i,j,k,m,0] =train[i].joints[j,k,m,0]
                train_data[i,j,k,m,1] =train[i].joints[j,k,m,1]
                train_data[i,j,k,m,2] =train[i].joints[j,k,m,2]
    one_hot_vector = np.zeros(60)
    one_hot_vector[train[i].label - 1] = 1
    train_label[i] = one_hot_vector
    
 


test_data = np.zeros(((((len(test),sequence_number,joint_number,2,3)))))
test_label = np.zeros((len(test),60))

for i in range(len(test)):
    for j in range(test[i].joints.shape[0]):
        for k in range(test[i].joints.shape[1]):
            for m in range(skeleton_number):
                test_data[i,j,k,m,0] =test[i].joints[j,k,m,0]
                test_data[i,j,k,m,1] =test[i].joints[j,k,m,1]
                test_data[i,j,k,m,2] =test[i].joints[j,k,m,2]
    one_hot_vector = np.zeros(60)
    one_hot_vector[test[i].label - 1] = 1
    test_label[i] = one_hot_vector
    
    
os.chdir("/home/murat/Desktop/ebbt_har/compact_data")    

pickle.dump(train_label, open('train_label', 'wb')) 
 
pickle.dump(test_label, open('test_label', 'wb')) 

FileNotFoundError: [Errno 2] No such file or directory: '/home/murat/Desktop/ebbt_har/compact_data'

In [33]:
os.chdir("/home/murat/Desktop/ebbt_har/compact_data")    

train_label = train_label.astype('float32')
test_label = test_label.astype('float32')
pickle.dump(train_label, open('train_label', 'wb')) 
pickle.dump(test_label, open('test_label', 'wb')) 

FileNotFoundError: [Errno 2] No such file or directory: '/home/murat/Desktop/ebbt_har/compact_data'

In [34]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
pickle.dump(train_data, open('train_data', 'wb'))
pickle.dump(test_data, open('test_data', 'wb'))

In [35]:
train_label[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [36]:
train_data.shape

(5000, 100, 25, 2, 3)

In [43]:
# A simple CNN to predict certain characteristics of the human subject from MRI images.
# 3d convolution is used in each layer.
# Reference: https://www.tensorflow.org/get_started/mnist/pros, http://blog.naver.com/kjpark79/220783765651
# Adjust needed for your dataset e.g., max pooling, convolution parameters, training_step, batch size, etc

width = train_data.shape[1]
height = train_data.shape[2]
depth = train_data.shape[3]
nLabel = 60

batch_size = 3


# Start TensorFlow InteractiveSession

import tensorflow as tf

sess = tf.InteractiveSession()





# Placeholders (MNIST image:28x28pixels=784, label=10)
x_image = tf.placeholder(tf.float32, shape=[None, width,height,depth,3]) # [None, 28*28]
y_ = tf.placeholder(tf.float32, shape=[None, nLabel])  # [None, 10]





#kernel_conv1 = tf.Variable(tf.truncated_normal([3, 3, 3, 3, 32], dtype=tf.float32, stddev=0.1))
#biases_conv1 = tf.Variable(tf.constant(0.1, shape=[32], dtype=tf.float32))
#kernel_conv2 = tf.Variable(tf.truncated_normal([3, 3, 3, 32, 64], dtype=tf.float32, stddev=0.1))
#biases_conv2 = tf.Variable(tf.constant(0.1, shape=[64], dtype=tf.float32))



## Weight Initialization
# Create lots of weights and biases & Initialize with a small positive number as we will use ReLU
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

## Convolution and Pooling
# Convolution here: stride=1, zero-padded -> output size = input size
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1, 1, 1, 1, 1], padding='SAME') # conv2d, [1, 1, 1, 1]

# Pooling: max pooling over 2x2 blocks
def max_pool_2x2(x):  # tf.nn.max_pool. ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1]
    return tf.nn.max_pool3d(x, ksize=[1, 2, 2, 2, 1], strides=[1, 1, 2, 2, 1], padding='SAME')





## First Convolutional Layer
# Conv then Max-pooling. 1st layer will have 32 features for each 5x5 patch. (1 feature -> 32 features)
W_conv1 = weight_variable([2, 2, 2, 3, 64])  # shape of weight tensor = [5,5,1,32]
b_conv1 = bias_variable([64])  # bias vector for each output channel. = [32]

# Reshape 'x' to a 4D tensor (2nd dim=image width, 3rd dim=image height, 4th dim=nColorChannel)
#x_image = tf.reshape(x, [-1,width,height,depth,3]) # [-1,28,28,1]
print(x_image.get_shape) # (?, 256, 256, 40, 1)  # -> output image: 28x28 x1

# x_image * weight tensor + bias -> apply ReLU -> apply max-pool
h_conv1 = tf.nn.relu(conv3d(x_image, W_conv1) + b_conv1)  # conv2d, ReLU(x_image * weight + bias)
print(h_conv1.get_shape) # (?, 256, 256, 40, 32)  # -> output image: 28x28 x32
h_pool1 = max_pool_2x2(h_conv1)  # apply max-pool 
print(h_pool1.get_shape) # (?, 128, 128, 20, 32)  # -> output image: 14x14 x32




## Second Convolutional Layer
# Conv then Max-pooling. 2nd layer will have 64 features for each 5x5 patch. (32 features -> 64 features)
W_conv2 = weight_variable([2, 2, 2, 64, 128]) # [5, 5, 32, 64]
b_conv2 = bias_variable([128]) # [64]

h_conv2 = tf.nn.relu(conv3d(h_pool1, W_conv2) + b_conv2)  # conv2d, .ReLU(x_image * weight + bias)
print(h_conv2.get_shape) # (?, 128, 128, 20, 64)  # -> output image: 14x14 x64
h_pool2 = max_pool_2x2(h_conv2)  # apply max-pool 
print(h_pool2.get_shape) # (?, 64, 64, 10, 64)    # -> output image: 7x7 x64




## Densely Connected Layer (or fully-connected layer)
# fully-connected layer with 1024 neurons to process on the entire image
W_fc1 = weight_variable([100*7*1*128, 1024])  # [7*7*64, 1024]
b_fc1 = bias_variable([1024]) # [1024]]

W_fc2 = weight_variable([1024, 512])  # [7*7*64, 1024]
b_fc2 = bias_variable([512]) # [1024]]

h_pool2_flat = tf.reshape(h_pool2, [-1, 100*7*1*128])  # -> output image: [-1, 7*7*64] = 3136
print(h_pool2_flat.get_shape)  # (?, 2621440)
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  # ReLU(h_pool2_flat x weight + bias)
print(h_fc1.get_shape) # (?, 1024)  # -> output: 1024

h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)  # ReLU(h_pool2_flat x weight + bias)
print(h_fc2.get_shape) # (?, 1024)  # -> output: 1024


## Dropout (to reduce overfitting; useful when training very large neural network)
# We will turn on dropout during training & turn off during testing
keep_prob = tf.placeholder(tf.float32)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)
print(h_fc2_drop.get_shape)  # -> output: 1024


## Readout Layer
W_fc3 = weight_variable([512, nLabel]) # [1024, 10]
b_fc3 = bias_variable([nLabel]) # [10]

y_conv = tf.matmul(h_fc2_drop, W_fc3) + b_fc3
print(y_conv.get_shape)  # -> output: 10



## Train and Evaluate the Model
# set up for optimization (optimizer:ADAM)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)  # 1e-4
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


<bound method Tensor.get_shape of <tf.Tensor 'Placeholder_6:0' shape=(?, 100, 25, 2, 3) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_8:0' shape=(?, 100, 25, 2, 64) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'MaxPool3D_4:0' shape=(?, 100, 13, 1, 64) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_9:0' shape=(?, 100, 13, 1, 128) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'MaxPool3D_5:0' shape=(?, 100, 7, 1, 128) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Reshape_8:0' shape=(?, 89600) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_10:0' shape=(?, 1024) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_11:0' shape=(?, 512) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'dropout_2/mul:0' shape=(?, 512) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'add_14:0' shape=(?, 60) dtype=float32>>


In [44]:
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

In [ ]:
sess.run(tf.global_variables_initializer())
print ("Variables was initialized")

num_steps = 100   #115 #258
epochs = 7 


for e in range(epochs):
    train_dataset_rand, train_labels_rand = randomize(np.array(train_data), train_label)
    for step in range(num_steps):
        offset = (step * batch_size)% (train_labels_rand.shape[0] - batch_size) 
        
        if step % 5 == 0:
            train_accuracy = accuracy.eval(feed_dict = {x_image : train_dataset_rand[offset:(offset + batch_size)], y_ : train_labels_rand[offset:(offset + batch_size)], keep_prob: 1.0})
            print("epoch %d, step %d, train set batch accuracy %.1f"%(e+1, step, train_accuracy * 100))
            
        if step % 15 ==0:
            valid_accuracy = accuracy.eval(feed_dict={x_image : test_data, y_ : test_label, keep_prob: 1.0})
            print ("Validation accuracy: %.1f%%" % (valid_accuracy * 100))
            
            
        train_step.run(feed_dict = {x_image : train_dataset_rand[offset:(offset + batch_size)], y_ : train_labels_rand[offset:(offset + batch_size)], keep_prob: 0.5})
            
print("TRAINING WAS COMPLETED!") 